### IMPORTS

In [1]:
# load, preprocess, scale, baseline
from wrangle import final_dataset

# manipulate data
import numpy as np

# visualize data
import matplotlib.pyplot as plt
import seaborn as sn

# neural network weights
from modelsNN import good_weights

# optmization problem
from modelsRO import good_problem

np.random.seed(123)

### Load, Clean, Preprocess, Scale, Baseline: Cardiovascular Disease

In [2]:
X_train_scaled_cd, X_test_scaled_cd, y_train_cd, y_test_cd = final_dataset(dataset='cvd')

CVD Loading and Cleaning...
CVD Loaded and Cleaned...

CVD Splitting...
CVD Split...

CVD Scaling...
CVD Scaled...



### Neural Network Comparisons: Cardiovascular Disease

In [3]:
%%time
nn_algo_tune = good_weights(X_train_scaled_cd, y_train_cd,
                            X_test_scaled_cd, y_test_cd)

Running Random Hill Climbing
Get_NN_Algo: Random Hill Climbing
Instantiated Random Hill Climbing
Time (Seconds): 3.910064697265625e-05

Working on: Random Hill Climbing
Fitting: Random Hill Climbing
Model Fitting Complete. Time: 32.2075355052948 seconds

Predicting (TRAINING): Random Hill Climbing

Calculating Recall (TRAINING): Random Hill Climbing

Appending Results (TRAINING): Random Hill Climbing

Recall Score (TRAINING): 0.5462026804608512%

Predicting (TESTING): Random Hill Climbing

Calculting Recall (TESTING): Random Hill Climbing

Appending Results (TESTING): Random Hill Climbing

Recall Score (TESTING): 0.5712383488681758%

Running Random Hill Climbing
Get_NN_Algo: Random Hill Climbing
Instantiated Random Hill Climbing
Time (Seconds): 1.811981201171875e-05

Working on: Random Hill Climbing
Fitting: Random Hill Climbing
Model Fitting Complete. Time: 64.73573207855225 seconds

Predicting (TRAINING): Random Hill Climbing

Calculating Recall (TRAINING): Random Hill Climbing

Appe

### Optimization Problem: Cardiovascular Disease

In [4]:
%%time
opt_prob = good_problem()

Running All Algorithms

Running Random Hill Climbing
Running Random Hill Climbing FourPeaks
Completed Random Hill Climbing FourPeaks. Time: 0.3187445640563965 minutes.
Running Random Hill Climbing FlipFlop
Completed Random Hill Climbing FlipFlop. Time: 0.32537227074305214 minutes.
Running Random Hill Climbing Knapsack
Completed Random Hill Climbing Knapsack. Time: 0.31682502031326293 minutes.
Completed Random Hill Climbing. Time: 0.9609418551127116 minutes

Running Random Hill Climbing
Running Random Hill Climbing FourPeaks
Completed Random Hill Climbing FourPeaks. Time: 0.3213028152783712 minutes.
Running Random Hill Climbing FlipFlop
Completed Random Hill Climbing FlipFlop. Time: 0.32963773409525554 minutes.
Running Random Hill Climbing Knapsack
Completed Random Hill Climbing Knapsack. Time: 0.32008928855260216 minutes.
Completed Random Hill Climbing. Time: 0.971029837926229 minutes

Running Random Hill Climbing
Running Random Hill Climbing FourPeaks
Completed Random Hill Climbing Fo

### Analysis: Neural Network Comparisons

In [ ]:
nn_algo_tune

### Analysis: Optimization Problem

$$
\textbf{Random Hill Climbing: Four Peaks}\\~\\
\textbf{Seed Average}
$$

In [ ]:
# seed 1

# boolean masks
bool1 = opt_prob['Random Hill Climbing']['rhc_combo1']['results'][1]['df_run_curves_FP']['Restarts'] == 15
bool2 = opt_prob['Random Hill Climbing']['rhc_combo2']['results'][1]['df_run_curves_FP']['Restarts'] == 15
bool3 = opt_prob['Random Hill Climbing']['rhc_combo3']['results'][1]['df_run_curves_FP']['Restarts'] == 15

rhc_combo1FP = (opt_prob['Random Hill Climbing']['rhc_combo1']['results'][1]['df_run_curves_FP'][bool1]
    .reset_index(drop=True))
rhc_combo2FP = (opt_prob['Random Hill Climbing']['rhc_combo2']['results'][1]['df_run_curves_FP'][bool2]
    .reset_index(drop=True))
rhc_combo3FP = (opt_prob['Random Hill Climbing']['rhc_combo3']['results'][1]['df_run_curves_FP'][bool3]
    .reset_index(drop=True))

# seed 2

# boolean masks
bool4 = opt_prob['Random Hill Climbing']['rhc_combo4']['results'][1]['df_run_curves_FP']['Restarts'] == 15
bool5 = opt_prob['Random Hill Climbing']['rhc_combo5']['results'][1]['df_run_curves_FP']['Restarts'] == 15
bool6 = opt_prob['Random Hill Climbing']['rhc_combo6']['results'][1]['df_run_curves_FP']['Restarts'] == 15

rhc_combo4FP = (opt_prob['Random Hill Climbing']['rhc_combo4']['results'][1]['df_run_curves_FP'][bool4]
    .reset_index(drop=True))
rhc_combo5FP = (opt_prob['Random Hill Climbing']['rhc_combo5']['results'][1]['df_run_curves_FP'][bool5]
    .reset_index(drop=True))
rhc_combo6FP = (opt_prob['Random Hill Climbing']['rhc_combo6']['results'][1]['df_run_curves_FP'][bool6]
    .reset_index(drop=True))

# seed 3

# boolean masks
bool7 = opt_prob['Random Hill Climbing']['rhc_combo7']['results'][1]['df_run_curves_FP']['Restarts'] == 15
bool8 = opt_prob['Random Hill Climbing']['rhc_combo8']['results'][1]['df_run_curves_FP']['Restarts'] == 15
bool9 = opt_prob['Random Hill Climbing']['rhc_combo9']['results'][1]['df_run_curves_FP']['Restarts'] == 15

rhc_combo7FP = (opt_prob['Random Hill Climbing']['rhc_combo7']['results'][1]['df_run_curves_FP'][bool7]
    .reset_index(drop=True))
rhc_combo8FP = (opt_prob['Random Hill Climbing']['rhc_combo8']['results'][1]['df_run_curves_FP'][bool8]
    .reset_index(drop=True))
rhc_combo9FP = (opt_prob['Random Hill Climbing']['rhc_combo9']['results'][1]['df_run_curves_FP'][bool9]
    .reset_index(drop=True))

# Average Seed
max_index_seed1 = max(rhc_combo1FP.index.max() + 1,
                      rhc_combo4FP.index.max() + 1,
                      rhc_combo7FP.index.max() + 1)

max_index_seed2 = max(rhc_combo2FP.index.max() + 1,
                      rhc_combo5FP.index.max() + 1,
                      rhc_combo8FP.index.max() + 1)

max_index_seed3 = max(rhc_combo3FP.index.max() + 1,
                      rhc_combo6FP.index.max() + 1,
                      rhc_combo9FP.index.max() + 1)

rhc_combo1FP = rhc_combo1FP.reindex(index=range(0, max_index_seed1),
                                    fill_value=0)
rhc_combo4FP = rhc_combo4FP.reindex(index=range(0, max_index_seed1),
                                    fill_value=0)
rhc_combo7FP = rhc_combo7FP.reindex(index=range(0, max_index_seed1),
                                    fill_value=0)

rhc_combo2FP = rhc_combo2FP.reindex(index=range(0, max_index_seed2),
                                    fill_value=0)
rhc_combo5FP = rhc_combo5FP.reindex(index=range(0, max_index_seed2),
                                    fill_value=0)
rhc_combo8FP = rhc_combo8FP.reindex(index=range(0, max_index_seed2),
                                    fill_value=0)

rhc_combo3FP = rhc_combo3FP.reindex(index=range(0, max_index_seed3),
                                    fill_value=0)
rhc_combo6FP = rhc_combo6FP.reindex(index=range(0, max_index_seed3),
                                    fill_value=0)
rhc_combo9FP = rhc_combo9FP.reindex(index=range(0, max_index_seed3),
                                    fill_value=0)

rhc_seed1avg = (rhc_combo1FP['Fitness'] + rhc_combo4FP['Fitness'] + rhc_combo7FP['Fitness']) / 3
rhc_seed2avg = (rhc_combo2FP['Fitness'] + rhc_combo5FP['Fitness'] + rhc_combo8FP['Fitness']) / 3
rhc_seed3avg = (rhc_combo3FP['Fitness'] + rhc_combo6FP['Fitness'] + rhc_combo9FP['Fitness']) / 3

In [ ]:
# Concatenate These

# Common Problem Size 1; Different Seeds
rhc_combo1FP
rhc_combo4FP
rhc_combo7FP

# Common Problem Size 2; Different Seeds
rhc_combo2FP
rhc_combo5FP
rhc_combo8FP

# Common Problem Size 3; Different Seeds
rhc_combo3FP
rhc_combo6FP
rhc_combo9FP

$$
\textbf{Random Hill Climbing: Flip Flop}\\~\\
\textbf{Seed Average}
$$

In [ ]:
# seed 1

# boolean masks
bool1 = opt_prob['Random Hill Climbing']['rhc_combo1']['results'][1]['df_run_curves_FF']['Restarts'] == 15
bool2 = opt_prob['Random Hill Climbing']['rhc_combo2']['results'][1]['df_run_curves_FF']['Restarts'] == 15
bool3 = opt_prob['Random Hill Climbing']['rhc_combo3']['results'][1]['df_run_curves_FF']['Restarts'] == 15

rhc_combo1FF = (opt_prob['Random Hill Climbing']['rhc_combo1']['results'][1]['df_run_curves_FF'][bool1]
    .reset_index(drop=True))
rhc_combo2FF = (opt_prob['Random Hill Climbing']['rhc_combo2']['results'][1]['df_run_curves_FF'][bool2]
    .reset_index(drop=True))
rhc_combo3FF = (opt_prob['Random Hill Climbing']['rhc_combo3']['results'][1]['df_run_curves_FF'][bool3]
    .reset_index(drop=True))

# seed 2

# boolean masks
bool4 = opt_prob['Random Hill Climbing']['rhc_combo4']['results'][1]['df_run_curves_FF']['Restarts'] == 15
bool5 = opt_prob['Random Hill Climbing']['rhc_combo5']['results'][1]['df_run_curves_FF']['Restarts'] == 15
bool6 = opt_prob['Random Hill Climbing']['rhc_combo6']['results'][1]['df_run_curves_FF']['Restarts'] == 15

rhc_combo4FF = (opt_prob['Random Hill Climbing']['rhc_combo4']['results'][1]['df_run_curves_FF'][bool4]
    .reset_index(drop=True))
rhc_combo5FF = (opt_prob['Random Hill Climbing']['rhc_combo5']['results'][1]['df_run_curves_FF'][bool5]
    .reset_index(drop=True))
rhc_combo6FF = (opt_prob['Random Hill Climbing']['rhc_combo6']['results'][1]['df_run_curves_FF'][bool6]
    .reset_index(drop=True))

# seed 3

# boolean masks
bool7 = opt_prob['Random Hill Climbing']['rhc_combo7']['results'][1]['df_run_curves_FF']['Restarts'] == 15
bool8 = opt_prob['Random Hill Climbing']['rhc_combo8']['results'][1]['df_run_curves_FF']['Restarts'] == 15
bool9 = opt_prob['Random Hill Climbing']['rhc_combo9']['results'][1]['df_run_curves_FF']['Restarts'] == 15

rhc_combo7FF = (opt_prob['Random Hill Climbing']['rhc_combo7']['results'][1]['df_run_curves_FF'][bool7]
    .reset_index(drop=True))
rhc_combo8FF = (opt_prob['Random Hill Climbing']['rhc_combo8']['results'][1]['df_run_curves_FF'][bool8]
    .reset_index(drop=True))
rhc_combo9FF = (opt_prob['Random Hill Climbing']['rhc_combo9']['results'][1]['df_run_curves_FF'][bool9]
    .reset_index(drop=True))

# Average Seed
max_index_seed1 = max(rhc_combo1FF.index.max() + 1,
                      rhc_combo4FF.index.max() + 1,
                      rhc_combo7FF.index.max() + 1)

max_index_seed2 = max(rhc_combo2FF.index.max() + 1,
                      rhc_combo5FF.index.max() + 1,
                      rhc_combo8FF.index.max() + 1)

max_index_seed3 = max(rhc_combo3FF.index.max() + 1,
                      rhc_combo6FF.index.max() + 1,
                      rhc_combo9FF.index.max() + 1)

rhc_combo1FF = rhc_combo1FF.reindex(index=range(0, max_index_seed1),
                                    fill_value=0)
rhc_combo4FF = rhc_combo4FF.reindex(index=range(0, max_index_seed1),
                                    fill_value=0)
rhc_combo7FF = rhc_combo7FF.reindex(index=range(0, max_index_seed1),
                                    fill_value=0)

rhc_combo2FF = rhc_combo2FF.reindex(index=range(0, max_index_seed2),
                                    fill_value=0)
rhc_combo5FF = rhc_combo5FF.reindex(index=range(0, max_index_seed2),
                                    fill_value=0)
rhc_combo8FF = rhc_combo8FF.reindex(index=range(0, max_index_seed2),
                                    fill_value=0)

rhc_combo3FF = rhc_combo3FF.reindex(index=range(0, max_index_seed3),
                                    fill_value=0)
rhc_combo6FF = rhc_combo6FF.reindex(index=range(0, max_index_seed3),
                                    fill_value=0)
rhc_combo9FF = rhc_combo9FF.reindex(index=range(0, max_index_seed3),
                                    fill_value=0)

rhc_seed1avg = (rhc_combo1FF['Fitness'] + rhc_combo4FF['Fitness'] + rhc_combo7FF['Fitness']) / 3
rhc_seed2avg = (rhc_combo2FF['Fitness'] + rhc_combo5FF['Fitness'] + rhc_combo8FF['Fitness']) / 3
rhc_seed3avg = (rhc_combo3FF['Fitness'] + rhc_combo6FF['Fitness'] + rhc_combo9FF['Fitness']) / 3

In [ ]:
# Concatenate These

# Common Problem Size 1; Different Seeds
rhc_combo1FF
rhc_combo4FF
rhc_combo7FF

# Common Problem Size 2; Different Seeds
rhc_combo2FF
rhc_combo5FF
rhc_combo8FF

# Common Problem Size 3; Different Seeds
rhc_combo3FF
rhc_combo6FF
rhc_combo9FF

$$
\textbf{Random Hill Climbing: Knapsack}\\~\\
\textbf{Seed Average}
$$

In [ ]:
# seed 1

# boolean masks
bool1 = opt_prob['Random Hill Climbing']['rhc_combo1']['results'][1]['df_run_curves_KS']['Restarts'] == 15
bool2 = opt_prob['Random Hill Climbing']['rhc_combo2']['results'][1]['df_run_curves_KS']['Restarts'] == 15
bool3 = opt_prob['Random Hill Climbing']['rhc_combo3']['results'][1]['df_run_curves_KS']['Restarts'] == 15

rhc_combo1KS = (opt_prob['Random Hill Climbing']['rhc_combo1']['results'][1]['df_run_curves_KS'][bool1]
    .reset_index(drop=True))
rhc_combo2KS = (opt_prob['Random Hill Climbing']['rhc_combo2']['results'][1]['df_run_curves_KS'][bool2]
    .reset_index(drop=True))
rhc_combo3KS = (opt_prob['Random Hill Climbing']['rhc_combo3']['results'][1]['df_run_curves_KS'][bool3]
    .reset_index(drop=True))

# seed 2

# boolean masks
bool4 = opt_prob['Random Hill Climbing']['rhc_combo4']['results'][1]['df_run_curves_KS']['Restarts'] == 15
bool5 = opt_prob['Random Hill Climbing']['rhc_combo5']['results'][1]['df_run_curves_KS']['Restarts'] == 15
bool6 = opt_prob['Random Hill Climbing']['rhc_combo6']['results'][1]['df_run_curves_KS']['Restarts'] == 15

rhc_combo4KS = (opt_prob['Random Hill Climbing']['rhc_combo4']['results'][1]['df_run_curves_KS'][bool4]
    .reset_index(drop=True))
rhc_combo5KS = (opt_prob['Random Hill Climbing']['rhc_combo5']['results'][1]['df_run_curves_KS'][bool5]
    .reset_index(drop=True))
rhc_combo6KS = (opt_prob['Random Hill Climbing']['rhc_combo6']['results'][1]['df_run_curves_KS'][bool6]
    .reset_index(drop=True))

# seed 3

# boolean masks
bool7 = opt_prob['Random Hill Climbing']['rhc_combo7']['results'][1]['df_run_curves_KS']['Restarts'] == 15
bool8 = opt_prob['Random Hill Climbing']['rhc_combo8']['results'][1]['df_run_curves_KS']['Restarts'] == 15
bool9 = opt_prob['Random Hill Climbing']['rhc_combo9']['results'][1]['df_run_curves_KS']['Restarts'] == 15

rhc_combo7KS = (opt_prob['Random Hill Climbing']['rhc_combo7']['results'][1]['df_run_curves_KS'][bool7]
    .reset_index(drop=True))
rhc_combo8KS = (opt_prob['Random Hill Climbing']['rhc_combo8']['results'][1]['df_run_curves_KS'][bool8]
    .reset_index(drop=True))
rhc_combo9KS = (opt_prob['Random Hill Climbing']['rhc_combo9']['results'][1]['df_run_curves_KS'][bool9]
    .reset_index(drop=True))

# Average Seed
max_index_seed1 = max(rhc_combo1KS.index.max() + 1,
                      rhc_combo4KS.index.max() + 1,
                      rhc_combo7KS.index.max() + 1)

max_index_seed2 = max(rhc_combo2KS.index.max() + 1,
                      rhc_combo5KS.index.max() + 1,
                      rhc_combo8KS.index.max() + 1)

max_index_seed3 = max(rhc_combo3KS.index.max() + 1,
                      rhc_combo6KS.index.max() + 1,
                      rhc_combo9KS.index.max() + 1)

rhc_combo1KS = rhc_combo1KS.reindex(index=range(0, max_index_seed1),
                                    fill_value=0)
rhc_combo4KS = rhc_combo4KS.reindex(index=range(0, max_index_seed1),
                                    fill_value=0)
rhc_combo7KS = rhc_combo7KS.reindex(index=range(0, max_index_seed1),
                                    fill_value=0)

rhc_combo2KS = rhc_combo2KS.reindex(index=range(0, max_index_seed2),
                                    fill_value=0)
rhc_combo5KS = rhc_combo5KS.reindex(index=range(0, max_index_seed2),
                                    fill_value=0)
rhc_combo8KS = rhc_combo8KS.reindex(index=range(0, max_index_seed2),
                                    fill_value=0)

rhc_combo3KS = rhc_combo3KS.reindex(index=range(0, max_index_seed3),
                                    fill_value=0)
rhc_combo6KS = rhc_combo6KS.reindex(index=range(0, max_index_seed3),
                                    fill_value=0)
rhc_combo9KS = rhc_combo9KS.reindex(index=range(0, max_index_seed3),
                                    fill_value=0)

rhc_seed1avg = (rhc_combo1KS['Fitness'] + rhc_combo4KS['Fitness'] + rhc_combo7KS['Fitness']) / 3
rhc_seed2avg = (rhc_combo2KS['Fitness'] + rhc_combo5KS['Fitness'] + rhc_combo8KS['Fitness']) / 3
rhc_seed3avg = (rhc_combo3KS['Fitness'] + rhc_combo6KS['Fitness'] + rhc_combo9KS['Fitness']) / 3

In [ ]:
# Concatenate These

# Common Problem Size 1; Different Seeds
rhc_combo1KS
rhc_combo4KS
rhc_combo7KS

# Common Problem Size 2; Different Seeds
rhc_combo2KS
rhc_combo5KS
rhc_combo8KS

# Common Problem Size 3; Different Seeds
rhc_combo3KS
rhc_combo6KS
rhc_combo9KS

$$
\textbf{Simulated Annleaing: Four Peaks}\\~\\
\textbf{Seed Average}
$$

In [ ]:

# seed 1
sa_combo1FP = (opt_prob['Simulated Annealing']['sa_combo1']['results'][1]['df_run_curves_FP']
    .reset_index(drop=True))
sa_combo2FP = (opt_prob['Simulated Annealing']['sa_combo2']['results'][1]['df_run_curves_FP']
    .reset_index(drop=True))
sa_combo3FP = (opt_prob['Simulated Annealing']['sa_combo3']['results'][1]['df_run_curves_FP']
    .reset_index(drop=True))

# seed 2
sa_combo4FP = (opt_prob['Simulated Annealing']['sa_combo4']['results'][1]['df_run_curves_FP']
    .reset_index(drop=True))
sa_combo5FP = (opt_prob['Simulated Annealing']['sa_combo5']['results'][1]['df_run_curves_FP']
    .reset_index(drop=True))
sa_combo6FP = (opt_prob['Simulated Annealing']['sa_combo6']['results'][1]['df_run_curves_FP']
    .reset_index(drop=True))

# seed 3
sa_combo7FP = (opt_prob['Simulated Annealing']['sa_combo7']['results'][1]['df_run_curves_FP']
    .reset_index(drop=True))
sa_combo8FP = (opt_prob['Simulated Annealing']['sa_combo8']['results'][1]['df_run_curves_FP']
    .reset_index(drop=True))
sa_combo9FP = (opt_prob['Simulated Annealing']['sa_combo9']['results'][1]['df_run_curves_FP']
    .reset_index(drop=True))

# Average Seed
max_index_seed1 = max(sa_combo1FP.index.max() + 1,
                      sa_combo4FP.index.max() + 1,
                      sa_combo7FP.index.max() + 1)

max_index_seed2 = max(sa_combo2FP.index.max() + 1,
                      sa_combo5FP.index.max() + 1,
                      sa_combo8FP.index.max() + 1)

max_index_seed3 = max(sa_combo3FP.index.max() + 1,
                      sa_combo6FP.index.max() + 1,
                      sa_combo9FP.index.max() + 1)

sa_combo1FP = sa_combo1FP.reindex(index=range(0, max_index_seed1),
                                    fill_value=0)
sa_combo4FP = sa_combo4FP.reindex(index=range(0, max_index_seed1),
                                    fill_value=0)
sa_combo7FP = sa_combo7FP.reindex(index=range(0, max_index_seed1),
                                    fill_value=0)

sa_combo2FP = sa_combo2FP.reindex(index=range(0, max_index_seed2),
                                    fill_value=0)
sa_combo5FP = sa_combo5FP.reindex(index=range(0, max_index_seed2),
                                    fill_value=0)
sa_combo8FP = sa_combo8FP.reindex(index=range(0, max_index_seed2),
                                    fill_value=0)

sa_combo3FP = sa_combo3FP.reindex(index=range(0, max_index_seed3),
                                    fill_value=0)
sa_combo6FP = sa_combo6FP.reindex(index=range(0, max_index_seed3),
                                    fill_value=0)
sa_combo9FP = sa_combo9FP.reindex(index=range(0, max_index_seed3),
                                    fill_value=0)

sa_seed1avg = (sa_combo1FP['Fitness'] + sa_combo4FP['Fitness'] + sa_combo7FP['Fitness']) / 3
sa_seed2avg = (sa_combo2FP['Fitness'] + sa_combo5FP['Fitness'] + sa_combo8FP['Fitness']) / 3
sa_seed3avg = (sa_combo3FP['Fitness'] + sa_combo6FP['Fitness'] + sa_combo9FP['Fitness']) / 3

In [ ]:
# Concatenate These

# Common Problem Size 1; Different Seeds
sa_combo1FP
sa_combo4FP
sa_combo7FP

# Common Problem Size 2; Different Seeds
sa_combo2FP
sa_combo5FP
sa_combo8FP

# Common Problem Size 3; Different Seeds
sa_combo3FP
sa_combo6FP
sa_combo9FP

$$
\textbf{Simulated Annleaing: Flip Flop}\\~\\
\textbf{Seed Average}
$$

In [ ]:

# seed 1
sa_combo1FF = (opt_prob['Simulated Annealing']['sa_combo1']['results'][1]['df_run_curves_FF']
    .reset_index(drop=True))
sa_combo2FF = (opt_prob['Simulated Annealing']['sa_combo2']['results'][1]['df_run_curves_FF']
    .reset_index(drop=True))
sa_combo3FF = (opt_prob['Simulated Annealing']['sa_combo3']['results'][1]['df_run_curves_FF']
    .reset_index(drop=True))

# seed 2
sa_combo4FF = (opt_prob['Simulated Annealing']['sa_combo4']['results'][1]['df_run_curves_FF']
    .reset_index(drop=True))
sa_combo5FF = (opt_prob['Simulated Annealing']['sa_combo5']['results'][1]['df_run_curves_FF']
    .reset_index(drop=True))
sa_combo6FF = (opt_prob['Simulated Annealing']['sa_combo6']['results'][1]['df_run_curves_FF']
    .reset_index(drop=True))

# seed 3
sa_combo7FF = (opt_prob['Simulated Annealing']['sa_combo7']['results'][1]['df_run_curves_FF']
    .reset_index(drop=True))
sa_combo8FF = (opt_prob['Simulated Annealing']['sa_combo8']['results'][1]['df_run_curves_FF']
    .reset_index(drop=True))
sa_combo9FF = (opt_prob['Simulated Annealing']['sa_combo9']['results'][1]['df_run_curves_FF']
    .reset_index(drop=True))

# Average Seed
max_index_seed1 = max(sa_combo1FF.index.max() + 1,
                      sa_combo4FF.index.max() + 1,
                      sa_combo7FF.index.max() + 1)

max_index_seed2 = max(sa_combo2FF.index.max() + 1,
                      sa_combo5FF.index.max() + 1,
                      sa_combo8FF.index.max() + 1)

max_index_seed3 = max(sa_combo3FF.index.max() + 1,
                      sa_combo6FF.index.max() + 1,
                      sa_combo9FF.index.max() + 1)

sa_combo1FF = sa_combo1FF.reindex(index=range(0, max_index_seed1),
                                    fill_value=0)
sa_combo4FF = sa_combo4FF.reindex(index=range(0, max_index_seed1),
                                    fill_value=0)
sa_combo7FF = sa_combo7FF.reindex(index=range(0, max_index_seed1),
                                    fill_value=0)

sa_combo2FF = sa_combo2FF.reindex(index=range(0, max_index_seed2),
                                    fill_value=0)
sa_combo5FF = sa_combo5FF.reindex(index=range(0, max_index_seed2),
                                    fill_value=0)
sa_combo8FF = sa_combo8FF.reindex(index=range(0, max_index_seed2),
                                    fill_value=0)

sa_combo3FF = sa_combo3FF.reindex(index=range(0, max_index_seed3),
                                    fill_value=0)
sa_combo6FF = sa_combo6FF.reindex(index=range(0, max_index_seed3),
                                    fill_value=0)
sa_combo9FF = sa_combo9FF.reindex(index=range(0, max_index_seed3),
                                    fill_value=0)

sa_seed1avg = (sa_combo1FF['Fitness'] + sa_combo4FF['Fitness'] + sa_combo7FF['Fitness']) / 3
sa_seed2avg = (sa_combo2FF['Fitness'] + sa_combo5FF['Fitness'] + sa_combo8FF['Fitness']) / 3
sa_seed3avg = (sa_combo3FF['Fitness'] + sa_combo6FF['Fitness'] + sa_combo9FF['Fitness']) / 3

In [ ]:
# Concatenate These

# Common Problem Size 1; Different Seeds
sa_combo1FF
sa_combo4FF
sa_combo7FF

# Common Problem Size 2; Different Seeds
sa_combo2FF
sa_combo5FF
sa_combo8FF

# Common Problem Size 3; Different Seeds
sa_combo3FF
sa_combo6FF
sa_combo9FF

$$
\textbf{Simulated Annleaning: Knapsack}\\~\\
\textbf{Seed Average}
$$

In [ ]:

# seed 1
sa_combo1KS = (opt_prob['Simulated Annealing']['sa_combo1']['results'][1]['df_run_curves_KS']
    .reset_index(drop=True))
sa_combo2KS = (opt_prob['Simulated Annealing']['sa_combo2']['results'][1]['df_run_curves_KS']
    .reset_index(drop=True))
sa_combo3KS = (opt_prob['Simulated Annealing']['sa_combo3']['results'][1]['df_run_curves_KS']
    .reset_index(drop=True))

# seed 2
sa_combo4KS = (opt_prob['Simulated Annealing']['sa_combo4']['results'][1]['df_run_curves_KS']
    .reset_index(drop=True))
sa_combo5KS = (opt_prob['Simulated Annealing']['sa_combo5']['results'][1]['df_run_curves_KS']
    .reset_index(drop=True))
sa_combo6KS = (opt_prob['Simulated Annealing']['sa_combo6']['results'][1]['df_run_curves_KS']
    .reset_index(drop=True))

# seed 3
sa_combo7KS = (opt_prob['Simulated Annealing']['sa_combo7']['results'][1]['df_run_curves_KS']
    .reset_index(drop=True))
sa_combo8KS = (opt_prob['Simulated Annealing']['sa_combo8']['results'][1]['df_run_curves_KS']
    .reset_index(drop=True))
sa_combo9KS = (opt_prob['Simulated Annealing']['sa_combo9']['results'][1]['df_run_curves_KS']
    .reset_index(drop=True))

# Average Seed
max_index_seed1 = max(sa_combo1KS.index.max() + 1,
                      sa_combo4KS.index.max() + 1,
                      sa_combo7KS.index.max() + 1)

max_index_seed2 = max(sa_combo2KS.index.max() + 1,
                      sa_combo5KS.index.max() + 1,
                      sa_combo8KS.index.max() + 1)

max_index_seed3 = max(sa_combo3KS.index.max() + 1,
                      sa_combo6KS.index.max() + 1,
                      sa_combo9KS.index.max() + 1)

sa_combo1KS = sa_combo1KS.reindex(index=range(0, max_index_seed1),
                                    fill_value=0)
sa_combo4KS = sa_combo4KS.reindex(index=range(0, max_index_seed1),
                                    fill_value=0)
sa_combo7KS = sa_combo7KS.reindex(index=range(0, max_index_seed1),
                                    fill_value=0)

sa_combo2KS = sa_combo2KS.reindex(index=range(0, max_index_seed2),
                                    fill_value=0)
sa_combo5KS = sa_combo5KS.reindex(index=range(0, max_index_seed2),
                                    fill_value=0)
sa_combo8KS = sa_combo8KS.reindex(index=range(0, max_index_seed2),
                                    fill_value=0)

sa_combo3KS = sa_combo3KS.reindex(index=range(0, max_index_seed3),
                                    fill_value=0)
sa_combo6KS = sa_combo6KS.reindex(index=range(0, max_index_seed3),
                                    fill_value=0)
sa_combo9KS = sa_combo9KS.reindex(index=range(0, max_index_seed3),
                                    fill_value=0)

sa_seed1avg = (sa_combo1KS['Fitness'] + sa_combo4KS['Fitness'] + sa_combo7KS['Fitness']) / 3
sa_seed2avg = (sa_combo2KS['Fitness'] + sa_combo5KS['Fitness'] + sa_combo8KS['Fitness']) / 3
sa_seed3avg = (sa_combo3KS['Fitness'] + sa_combo6KS['Fitness'] + sa_combo9KS['Fitness']) / 3

In [ ]:
# Concatenate These

# Common Problem Size 1; Different Seeds
sa_combo1KS
sa_combo4KS
sa_combo7KS

# Common Problem Size 2; Different Seeds
sa_combo2KS
sa_combo5KS
sa_combo8KS

# Common Problem Size 3; Different Seeds
sa_combo3KS
sa_combo6KS
sa_combo9KS

$$
\textbf{Genetic Algorithm: Four Peaks}\\~\\
\textbf{Seed Average}
$$

In [ ]:

# seed 1
ga_combo1FP = (opt_prob['Genetic Algorithm']['ga_combo1']['results'][1]['df_run_curves_FP']
    .reset_index(drop=True))
ga_combo2FP = (opt_prob['Genetic Algorithm']['ga_combo2']['results'][1]['df_run_curves_FP']
    .reset_index(drop=True))
ga_combo3FP = (opt_prob['Genetic Algorithm']['ga_combo3']['results'][1]['df_run_curves_FP']
    .reset_index(drop=True))

# seed 2
ga_combo4FP = (opt_prob['Genetic Algorithm']['ga_combo4']['results'][1]['df_run_curves_FP']
    .reset_index(drop=True))
ga_combo5FP = (opt_prob['Genetic Algorithm']['ga_combo5']['results'][1]['df_run_curves_FP']
    .reset_index(drop=True))
ga_combo6FP = (opt_prob['Genetic Algorithm']['ga_combo6']['results'][1]['df_run_curves_FP']
    .reset_index(drop=True))

# seed 3
ga_combo7FP = (opt_prob['Genetic Algorithm']['ga_combo7']['results'][1]['df_run_curves_FP']
    .reset_index(drop=True))
ga_combo8FP = (opt_prob['Genetic Algorithm']['ga_combo8']['results'][1]['df_run_curves_FP']
    .reset_index(drop=True))
ga_combo9FP = (opt_prob['Genetic Algorithm']['ga_combo9']['results'][1]['df_run_curves_FP']
    .reset_index(drop=True))

# Average Seed
max_index_seed1 = max(ga_combo1FP.index.max() + 1,
                      ga_combo4FP.index.max() + 1,
                      ga_combo7FP.index.max() + 1)

max_index_seed2 = max(ga_combo2FP.index.max() + 1,
                      ga_combo5FP.index.max() + 1,
                      ga_combo8FP.index.max() + 1)

max_index_seed3 = max(ga_combo3FP.index.max() + 1,
                      ga_combo6FP.index.max() + 1,
                      ga_combo9FP.index.max() + 1)

ga_combo1FP = ga_combo1FP.reindex(index=range(0, max_index_seed1),
                                    fill_value=0)
ga_combo4FP = ga_combo4FP.reindex(index=range(0, max_index_seed1),
                                    fill_value=0)
ga_combo7FP = ga_combo7FP.reindex(index=range(0, max_index_seed1),
                                    fill_value=0)

ga_combo2FP = ga_combo2FP.reindex(index=range(0, max_index_seed2),
                                    fill_value=0)
ga_combo5FP = ga_combo5FP.reindex(index=range(0, max_index_seed2),
                                    fill_value=0)
ga_combo8FP = ga_combo8FP.reindex(index=range(0, max_index_seed2),
                                    fill_value=0)

ga_combo3FP = ga_combo3FP.reindex(index=range(0, max_index_seed3),
                                    fill_value=0)
ga_combo6FP = ga_combo6FP.reindex(index=range(0, max_index_seed3),
                                    fill_value=0)
ga_combo9FP = ga_combo9FP.reindex(index=range(0, max_index_seed3),
                                    fill_value=0)

ga_seed1avg = (ga_combo1FP['Fitness'] + ga_combo4FP['Fitness'] + ga_combo7FP['Fitness']) / 3
ga_seed2avg = (ga_combo2FP['Fitness'] + ga_combo5FP['Fitness'] + ga_combo8FP['Fitness']) / 3
ga_seed3avg = (ga_combo3FP['Fitness'] + ga_combo6FP['Fitness'] + ga_combo9FP['Fitness']) / 3

In [ ]:
# Concatenate These

# Common Problem Size 1; Different Seeds
ga_combo1FP
ga_combo4FP
ga_combo7FP

# Common Problem Size 2; Different Seeds
ga_combo2FP
ga_combo5FP
ga_combo8FP

# Common Problem Size 3; Different Seeds
ga_combo3FP
ga_combo6FP
ga_combo9FP

$$
\textbf{Genetic Algorithm: Flip Flop}\\~\\
\textbf{Seed Average}
$$

In [ ]:

# seed 1
ga_combo1FF = (opt_prob['Genetic Algorithm']['ga_combo1']['results'][1]['df_run_curves_FF']
    .reset_index(drop=True))
ga_combo2FF = (opt_prob['Genetic Algorithm']['ga_combo2']['results'][1]['df_run_curves_FF']
    .reset_index(drop=True))
ga_combo3FF = (opt_prob['Genetic Algorithm']['ga_combo3']['results'][1]['df_run_curves_FF']
    .reset_index(drop=True))

# seed 2
ga_combo4FF = (opt_prob['Genetic Algorithm']['ga_combo4']['results'][1]['df_run_curves_FF']
    .reset_index(drop=True))
ga_combo5FF = (opt_prob['Genetic Algorithm']['ga_combo5']['results'][1]['df_run_curves_FF']
    .reset_index(drop=True))
ga_combo6FF = (opt_prob['Genetic Algorithm']['ga_combo6']['results'][1]['df_run_curves_FF']
    .reset_index(drop=True))

# seed 3
ga_combo7FF = (opt_prob['Genetic Algorithm']['ga_combo7']['results'][1]['df_run_curves_FF']
    .reset_index(drop=True))
ga_combo8FF = (opt_prob['Genetic Algorithm']['ga_combo8']['results'][1]['df_run_curves_FF']
    .reset_index(drop=True))
ga_combo9FF = (opt_prob['Genetic Algorithm']['ga_combo9']['results'][1]['df_run_curves_FF']
    .reset_index(drop=True))

# Average Seed
max_index_seed1 = max(ga_combo1FF.index.max() + 1,
                      ga_combo4FF.index.max() + 1,
                      ga_combo7FF.index.max() + 1)

max_index_seed2 = max(ga_combo2FF.index.max() + 1,
                      ga_combo5FF.index.max() + 1,
                      ga_combo8FF.index.max() + 1)

max_index_seed3 = max(ga_combo3FF.index.max() + 1,
                      ga_combo6FF.index.max() + 1,
                      ga_combo9FF.index.max() + 1)

ga_combo1FF = ga_combo1FF.reindex(index=range(0, max_index_seed1),
                                    fill_value=0)
ga_combo4FF = ga_combo4FF.reindex(index=range(0, max_index_seed1),
                                    fill_value=0)
ga_combo7FF = ga_combo7FF.reindex(index=range(0, max_index_seed1),
                                    fill_value=0)

ga_combo2FF = ga_combo2FF.reindex(index=range(0, max_index_seed2),
                                    fill_value=0)
ga_combo5FF = ga_combo5FF.reindex(index=range(0, max_index_seed2),
                                    fill_value=0)
ga_combo8FF = ga_combo8FF.reindex(index=range(0, max_index_seed2),
                                    fill_value=0)

ga_combo3FF = ga_combo3FF.reindex(index=range(0, max_index_seed3),
                                    fill_value=0)
ga_combo6FF = ga_combo6FF.reindex(index=range(0, max_index_seed3),
                                    fill_value=0)
ga_combo9FF = ga_combo9FF.reindex(index=range(0, max_index_seed3),
                                    fill_value=0)

ga_seed1avg = (ga_combo1FF['Fitness'] + ga_combo4FF['Fitness'] + ga_combo7FF['Fitness']) / 3
ga_seed2avg = (ga_combo2FF['Fitness'] + ga_combo5FF['Fitness'] + ga_combo8FF['Fitness']) / 3
ga_seed3avg = (ga_combo3FF['Fitness'] + ga_combo6FF['Fitness'] + ga_combo9FF['Fitness']) / 3

In [ ]:
# Concatenate These

# Common Problem Size 1; Different Seeds
ga_combo1FF
ga_combo4FF
ga_combo7FF

# Common Problem Size 2; Different Seeds
ga_combo2FF
ga_combo5FF
ga_combo8FF

# Common Problem Size 3; Different Seeds
ga_combo3FF
ga_combo6FF
ga_combo9FF

$$
\textbf{Genetic Algorithm: Knapsack}\\~\\
\textbf{Seed Average}
$$

In [ ]:

# seed 1
ga_combo1KS = (opt_prob['Genetic Algorithm']['ga_combo1']['results'][1]['df_run_curves_KS']
    .reset_index(drop=True))
ga_combo2KS = (opt_prob['Genetic Algorithm']['ga_combo2']['results'][1]['df_run_curves_KS']
    .reset_index(drop=True))
ga_combo3KS = (opt_prob['Genetic Algorithm']['ga_combo3']['results'][1]['df_run_curves_KS']
    .reset_index(drop=True))

# seed 2
ga_combo4KS = (opt_prob['Genetic Algorithm']['ga_combo4']['results'][1]['df_run_curves_KS']
    .reset_index(drop=True))
ga_combo5KS = (opt_prob['Genetic Algorithm']['ga_combo5']['results'][1]['df_run_curves_KS']
    .reset_index(drop=True))
ga_combo6KS = (opt_prob['Genetic Algorithm']['ga_combo6']['results'][1]['df_run_curves_KS']
    .reset_index(drop=True))

# seed 3
ga_combo7KS = (opt_prob['Genetic Algorithm']['ga_combo7']['results'][1]['df_run_curves_KS']
    .reset_index(drop=True))
ga_combo8KS = (opt_prob['Genetic Algorithm']['ga_combo8']['results'][1]['df_run_curves_KS']
    .reset_index(drop=True))
ga_combo9KS = (opt_prob['Genetic Algorithm']['ga_combo9']['results'][1]['df_run_curves_KS']
    .reset_index(drop=True))

# Average Seed
max_index_seed1 = max(ga_combo1KS.index.max() + 1,
                      ga_combo4KS.index.max() + 1,
                      ga_combo7KS.index.max() + 1)

max_index_seed2 = max(ga_combo2KS.index.max() + 1,
                      ga_combo5KS.index.max() + 1,
                      ga_combo8KS.index.max() + 1)

max_index_seed3 = max(ga_combo3KS.index.max() + 1,
                      ga_combo6KS.index.max() + 1,
                      ga_combo9KS.index.max() + 1)

ga_combo1KS = ga_combo1KS.reindex(index=range(0, max_index_seed1),
                                    fill_value=0)
ga_combo4KS = ga_combo4KS.reindex(index=range(0, max_index_seed1),
                                    fill_value=0)
ga_combo7KS = ga_combo7KS.reindex(index=range(0, max_index_seed1),
                                    fill_value=0)

ga_combo2KS = ga_combo2KS.reindex(index=range(0, max_index_seed2),
                                    fill_value=0)
ga_combo5KS = ga_combo5KS.reindex(index=range(0, max_index_seed2),
                                    fill_value=0)
ga_combo8KS = ga_combo8KS.reindex(index=range(0, max_index_seed2),
                                    fill_value=0)

ga_combo3KS = ga_combo3KS.reindex(index=range(0, max_index_seed3),
                                    fill_value=0)
ga_combo6KS = ga_combo6KS.reindex(index=range(0, max_index_seed3),
                                    fill_value=0)
ga_combo9KS = ga_combo9KS.reindex(index=range(0, max_index_seed3),
                                    fill_value=0)

ga_seed1avg = (ga_combo1KS['Fitness'] + ga_combo4KS['Fitness'] + ga_combo7KS['Fitness']) / 3
ga_seed2avg = (ga_combo2KS['Fitness'] + ga_combo5KS['Fitness'] + ga_combo8KS['Fitness']) / 3
ga_seed3avg = (ga_combo3KS['Fitness'] + ga_combo6KS['Fitness'] + ga_combo9KS['Fitness']) / 3

In [ ]:
# Concatenate These

# Common Problem Size 1; Different Seeds
ga_combo1KS
ga_combo4KS
ga_combo7KS

# Common Problem Size 2; Different Seeds
ga_combo2KS
ga_combo5KS
ga_combo8KS

# Common Problem Size 3; Different Seeds
ga_combo3KS
ga_combo6KS
ga_combo9KS

$$
\textbf{Mutual-Information-Maximizing Input Clustering: Four Peaks}\\~\\
\textbf{Seed Average}
$$

In [ ]:

# seed 1
mimic_combo1FP = (opt_prob['MIMIC']['mimic_combo1']['results'][1]['df_run_curves_FP']
    .reset_index(drop=True))
mimic_combo2FP = (opt_prob['MIMIC']['mimic_combo2']['results'][1]['df_run_curves_FP']
    .reset_index(drop=True))
mimic_combo3FP = (opt_prob['MIMIC']['mimic_combo3']['results'][1]['df_run_curves_FP']
    .reset_index(drop=True))

# seed 2
mimic_combo4FP = (opt_prob['MIMIC']['mmiic_combo4']['results'][1]['df_run_curves_FP']
    .reset_index(drop=True))
mimic_combo5FP = (opt_prob['MIMIC']['mimic_combo5']['results'][1]['df_run_curves_FP']
    .reset_index(drop=True))
mimic_combo6FP = (opt_prob['MIMIC']['mmiic_combo6']['results'][1]['df_run_curves_FP']
    .reset_index(drop=True))

# seed 3
mimic_combo7FP = (opt_prob['MIMIC']['mimic_combo7']['results'][1]['df_run_curves_FP']
    .reset_index(drop=True))
mimic_combo8FP = (opt_prob['MIMIC']['mimic_combo8']['results'][1]['df_run_curves_FP']
    .reset_index(drop=True))
mimic_combo9FP = (opt_prob['MIMIC']['mimic_combo9']['results'][1]['df_run_curves_FP']
    .reset_index(drop=True))

# Average Seed
max_index_seed1 = max(mimic_combo1FP.index.max() + 1,
                      mimic_combo4FP.index.max() + 1,
                      mimic_combo7FP.index.max() + 1)

max_index_seed2 = max(mimic_combo2FP.index.max() + 1,
                      mimic_combo5FP.index.max() + 1,
                      mimic_combo8FP.index.max() + 1)

max_index_seed3 = max(mimic_combo3FP.index.max() + 1,
                      mimic_combo6FP.index.max() + 1,
                      mimic_combo9FP.index.max() + 1)

mimic_combo1FP = mimic_combo1FP.reindex(index=range(0, max_index_seed1),
                                        fill_value=0)   
mimic_combo4FP = mimic_combo4FP.reindex(index=range(0, max_index_seed1),
                                        fill_value=0)   
mimic_combo7FP = mimic_combo7FP.reindex(index=range(0, max_index_seed1),
                                        fill_value=0)   

mimic_combo2FP = mimic_combo2FP.reindex(index=range(0, max_index_seed2),
                                        fill_value=0)   
mimic_combo5FP = mimic_combo5FP.reindex(index=range(0, max_index_seed2),
                                        fill_value=0)   
mimic_combo8FP = mimic_combo8FP.reindex(index=range(0, max_index_seed2),
                                        fill_value=0)   

mimic_combo3FP = mimic_combo3FP.reindex(index=range(0, max_index_seed3),
                                        fill_value=0)
mimic_combo6FP = mimic_combo6FP.reindex(index=range(0, max_index_seed3),
                                        fill_value=0)
mimic_combo9FP = mimic_combo9FP.reindex(index=range(0, max_index_seed3),
                                        fill_value=0)

mimic_seed1avg = (mimic_combo1FP['Fitness'] + mimic_combo4FP['Fitness'] + mimic_combo7FP['Fitness']) / 3
mimic_seed2avg = (mimic_combo2FP['Fitness'] + mimic_combo5FP['Fitness'] + mimic_combo8FP['Fitness']) / 3
mimic_seed3avg = (mimic_combo3FP['Fitness'] + mimic_combo6FP['Fitness'] + mimic_combo9FP['Fitness']) / 3

In [ ]:
# Concatenate These

# Common Problem Size 1; Different Seeds
mimic_combo1FP
mimic_combo4FP
mimic_combo7FP

# Common Problem Size 2; Different Seeds
mimic_combo2FP
mimic_combo5FP
mimic_combo8FP

# Common Problem Size 3; Different Seeds
mimic_combo3FP
mimic_combo6FP
mimic_combo9FP

$$
\textbf{Mutual-Information-Maximizing Input Clustering: Flip Flop}\\~\\
\textbf{Seed Average}
$$

In [ ]:

# seed 1
mimic_combo1FF = (opt_prob['MIMIC']['mimic_combo1']['results'][1]['df_run_curves_FF']
    .reset_index(drop=True))
mimic_combo2FF = (opt_prob['MIMIC']['mimic_combo2']['results'][1]['df_run_curves_FF']
    .reset_index(drop=True))
mimic_combo3FF = (opt_prob['MIMIC']['mimic_combo3']['results'][1]['df_run_curves_FF']
    .reset_index(drop=True))

# seed 2
mimic_combo4FF = (opt_prob['MIMIC']['mmiic_combo4']['results'][1]['df_run_curves_FF']
    .reset_index(drop=True))
mimic_combo5FF = (opt_prob['MIMIC']['mimic_combo5']['results'][1]['df_run_curves_FF']
    .reset_index(drop=True))
mimic_combo6FF = (opt_prob['MIMIC']['mmiic_combo6']['results'][1]['df_run_curves_FF']
    .reset_index(drop=True))

# seed 3
mimic_combo7FF = (opt_prob['MIMIC']['mimic_combo7']['results'][1]['df_run_curves_FF']
    .reset_index(drop=True))
mimic_combo8FF = (opt_prob['MIMIC']['mimic_combo8']['results'][1]['df_run_curves_FF']
    .reset_index(drop=True))
mimic_combo9FF = (opt_prob['MIMIC']['mimic_combo9']['results'][1]['df_run_curves_FF']
    .reset_index(drop=True))

# Average Seed
max_index_seed1 = max(mimic_combo1FF.index.max() + 1,
                      mimic_combo4FF.index.max() + 1,
                      mimic_combo7FF.index.max() + 1)

max_index_seed2 = max(mimic_combo2FF.index.max() + 1,
                      mimic_combo5FF.index.max() + 1,
                      mimic_combo8FF.index.max() + 1)

max_index_seed3 = max(mimic_combo3FF.index.max() + 1,
                      mimic_combo6FF.index.max() + 1,
                      mimic_combo9FF.index.max() + 1)

mimic_combo1FF = mimic_combo1FF.reindex(index=range(0, max_index_seed1),
                                        fill_value=0)
mimic_combo4FF = mimic_combo4FF.reindex(index=range(0, max_index_seed1),
                                        fill_value=0)
mimic_combo7FF = mimic_combo7FF.reindex(index=range(0, max_index_seed1),
                                        fill_value=0)

mimic_combo2FF = mimic_combo2FF.reindex(index=range(0, max_index_seed2),
                                        fill_value=0)
mimic_combo5FF = mimic_combo5FF.reindex(index=range(0, max_index_seed2),
                                        fill_value=0)
mimic_combo8FF = mimic_combo8FF.reindex(index=range(0, max_index_seed2),
                                        fill_value=0)

mimic_combo3FF = mimic_combo3FF.reindex(index=range(0, max_index_seed3),
                                        fill_value=0)
mimic_combo6FF = mimic_combo6FF.reindex(index=range(0, max_index_seed3),
                                        fill_value=0)
mimic_combo9FF = mimic_combo9FF.reindex(index=range(0, max_index_seed3),
                                        fill_value=0)

mimic_seed1avg = (mimic_combo1FF['Fitness'] + mimic_combo4FF['Fitness'] + mimic_combo7FF['Fitness']) / 3
mimic_seed2avg = (mimic_combo2FF['Fitness'] + mimic_combo5FF['Fitness'] + mimic_combo8FF['Fitness']) / 3
mimic_seed3avg = (mimic_combo3FF['Fitness'] + mimic_combo6FF['Fitness'] + mimic_combo9FF['Fitness']) / 3

In [ ]:
# Concatenate These

# Common Problem Size 1; Different Seeds
mimic_combo1FF
mimic_combo4FF
mimic_combo7FF

# Common Problem Size 2; Different Seeds
mimic_combo2FF
mimic_combo5FF
mimic_combo8FF

# Common Problem Size 3; Different Seeds
mimic_combo3FF
mimic_combo6FF
mimic_combo9FF

$$
\textbf{Mutual-Information-Maximizing Input Clustering: Knapsack}\\~\\
\textbf{Seed Average}
$$

In [ ]:

# seed 1
mimic_combo1KS = (opt_prob['MIMIC']['mimic_combo1']['results'][1]['df_run_curves_KS']
    .reset_index(drop=True))
mimic_combo2KS = (opt_prob['MIMIC']['mimic_combo2']['results'][1]['df_run_curves_KS']
    .reset_index(drop=True))
mimic_combo3KS = (opt_prob['MIMIC']['mimic_combo3']['results'][1]['df_run_curves_KS']
    .reset_index(drop=True))

# seed 2
mimic_combo4KS = (opt_prob['MIMIC']['mmiic_combo4']['results'][1]['df_run_curves_KS']
    .reset_index(drop=True))
mimic_combo5KS = (opt_prob['MIMIC']['mimic_combo5']['results'][1]['df_run_curves_KS']
    .reset_index(drop=True))
mimic_combo6KS = (opt_prob['MIMIC']['mmiic_combo6']['results'][1]['df_run_curves_KS']
    .reset_index(drop=True))

# seed 3
mimic_combo7KS = (opt_prob['MIMIC']['mimic_combo7']['results'][1]['df_run_curves_KS']
    .reset_index(drop=True))
mimic_combo8KS = (opt_prob['MIMIC']['mimic_combo8']['results'][1]['df_run_curves_KS']
    .reset_index(drop=True))
mimic_combo9KS = (opt_prob['MIMIC']['mimic_combo9']['results'][1]['df_run_curves_KS']
    .reset_index(drop=True))

# Average Seed
max_index_seed1 = max(mimic_combo1KS.index.max() + 1,
                      mimic_combo4KS.index.max() + 1,
                      mimic_combo7KS.index.max() + 1)

max_index_seed2 = max(mimic_combo2KS.index.max() + 1,
                      mimic_combo5KS.index.max() + 1,
                      mimic_combo8KS.index.max() + 1)

max_index_seed3 = max(mimic_combo3KS.index.max() + 1,
                      mimic_combo6KS.index.max() + 1,
                      mimic_combo9KS.index.max() + 1)

mimic_combo1KS = mimic_combo1KS.reindex(index=range(0, max_index_seed1),
                                        fill_value=0)
mimic_combo4KS = mimic_combo4KS.reindex(index=range(0, max_index_seed1),
                                        fill_value=0)
mimic_combo7KS = mimic_combo7KS.reindex(index=range(0, max_index_seed1),
                                        fill_value=0)

mimic_combo2KS = mimic_combo2KS.reindex(index=range(0, max_index_seed2),
                                        fill_value=0)
mimic_combo5KS = mimic_combo5KS.reindex(index=range(0, max_index_seed2),
                                        fill_value=0)
mimic_combo8KS = mimic_combo8KS.reindex(index=range(0, max_index_seed2),
                                        fill_value=0)

mimic_combo3KS = mimic_combo3KS.reindex(index=range(0, max_index_seed3),
                                        fill_value=0)
mimic_combo6KS = mimic_combo6KS.reindex(index=range(0, max_index_seed3),
                                        fill_value=0)
mimic_combo9KS = mimic_combo9KS.reindex(index=range(0, max_index_seed3),
                                        fill_value=0)

mimic_seed1avg = (mimic_combo1KS['Fitness'] + mimic_combo4KS['Fitness'] + mimic_combo7KS['Fitness']) / 3
mimic_seed2avg = (mimic_combo2KS['Fitness'] + mimic_combo5KS['Fitness'] + mimic_combo8KS['Fitness']) / 3
mimic_seed3avg = (mimic_combo3KS['Fitness'] + mimic_combo6KS['Fitness'] + mimic_combo9KS['Fitness']) / 3

In [ ]:
# Concatenate These

# Common Problem Size 1; Different Seeds
mimic_combo1KS.reindex(index=range(0, max_index_seed1),
                        fill_value=0)
mimic_combo4KS.reindex(index=range(0, max_index_seed1),
                        fill_value=0)
mimic_combo7KS.reindex(index=range(0, max_index_seed1),
                        fill_value=0)

# Common Problem Size 2; Different Seeds
mimic_combo2KS.reindex(index=range(0, max_index_seed2),
                        fill_value=0)
mimic_combo5KS.reindex(index=range(0, max_index_seed2),
                        fill_value=0)
mimic_combo8KS.reindex(index=range(0, max_index_seed2),
                        fill_value=0)

# Common Problem Size 3; Different Seeds
mimic_combo3KS.reindex(index=range(0, max_index_seed3),
                        fill_value=0)
mimic_combo6KS.reindex(index=range(0, max_index_seed3),
                        fill_value=0)
mimic_combo9KS.reindex(index=range(0, max_index_seed3),
                        fill_value=0)

### Plotting

$$ \text{Plot RHC Seed Averages}
$$

In [ ]:
# FOUR PEAKS
# fig, axes = plt.subplots(nrows=1,
#                          ncols=3,
#                          figsize=(15, 5))
# for ax in axes:
#     ax.set_xlabel(xlabel='Iteration')
#     ax.set_ylabel(ylabel='Fitness')
#     title = f'RHC; Seed: 1; Size: 4; Four Peaks'
#     ax.set_title(label=title)
# plt.show()

# for i in range(1, i+1):
#     rhcFP1 = rhc_combo1FP[(rhc_combo1FP['current_restart'] == j) & (rhc_combo1FP['Restarts'] == 15)]        
#     plt.plot(rhcFP1['Iteration'], rhcFP1['Fitness'])
#     plt.xlabel(xlabel='Iteration')
#     plt.ylabel(ylabel='Fitness')
#     title = f'RHC; Seed: 1; Size: 4; Four Peaks; Restarts: {i}; Current Restart: {j}'
#     plt.title(label=title)
#     plt.show()

# combo 2
# for i in [5, 10, 15]:
#     for j in range(1, i+1):
#         rhcFP2 = rhc_combo2FP[(rhc_combo2FP['current_restart'] == j) & (rhc_combo2FP['Restarts'] == i)]        
#         plt.plot(rhcFP2['Iteration'], rhcFP2['Fitness'])
#         plt.xlabel(xlabel='Iteration')
#         plt.ylabel(ylabel='Fitness')
#         title = f'RHC; Seed: 1; Size: 6; Four Peaks; Restarts: {i}; Current Restart: {j}'
#         plt.title(label=title)
#         plt.show()

# combo 3
# for i in range(1, i+1):
#     rhcFP3 = rhc_combo3FP[(rhc_combo3FP['current_restart'] == j) & (rhc_combo3FP['Restarts'] == 15)]
#     plt.plot(rhcFP3['Iteration'], rhcFP3['Fitness'])
#     plt.xlabel(xlabel='Iteration')
#     plt.ylabel(ylabel='Fitness')
#     title = f'RHC; Seed: 1; Size: 8; Four Peaks; Restarts: {i}; Current Restart: {j}'
#     plt.title(label=title)
#     plt.show()

# FLIP FLOP
# Combo 1
# for i in range(15):
#     rhcFF1 = rhc_combo1FF[(rhc_combo1FF['current_restart'] == i) & (rhc_combo1FF['Restarts'] == 15)]
#     plt.plot(rhcFF1['Iteration'], rhcFF1['Fitness'])
#     plt.xlabel(xlabel='Iteration')
#     plt.ylabel(ylabel='Fitness')
#     title = f'RHC; Seed: 1; Size: 4; Flip Flop; Restarts: 15; Current Restart: {i}'
#     plt.title(label=title)
#     plt.show()

# Combo 2
# for i in [5, 10, 15]:
#     for j in range(1, i+1):
#         rhcFF2 = rhc_combo2FF[(rhc_combo2FF['current_restart'] == j) & (rhc_combo2FF['Restarts'] == i)]
#         plt.plot(rhcFF2['Iteration'], rhcFF2['Fitness'])
#         plt.xlabel(xlabel='Iteration')
#         plt.ylabel(ylabel='Fitness')
#         title = f'RHC; Seed: 1; Size: 6; Flip Flop; Restarts: {i}; Current Restart: {j}'
#         plt.title(label=title)
#         plt.show()

# Combo 3
# for i in [5, 10, 15]:
#     for j in range(1, i+1):
#         rhcFF3 = rhc_combo3FF[(rhc_combo3FF['current_restart'] == j) & (rhc_combo3FF['Restarts'] == i)]
#         plt.plot(rhcFF3['Iteration'], rhcFF3['Fitness'])
#         plt.xlabel(xlabel='Iteration')
#         plt.ylabel(ylabel='Fitness')
#         title = f'RHC; Seed: 1; Size: 8; Flip Flop; Restarts: {i}; Current Restart: {j}'
#         plt.title(label=title)
#         plt.show()

# KNAPSACK
# Combo 1
# for i in [5, 10, 15]:
#     for j in range(1, i+1):
#         rhcKS1 = rhc_combo1KS[(rhc_combo1KS['current_restart'] == j) & (rhc_combo1KS['Restarts'] == i)]
#         plt.plot(rhcKS1['Iteration'], rhcKS1['Fitness'])
#         plt.xlabel(xlabel='Iteration')
#         plt.ylabel(ylabel='Fitness')
#         title = f'RHC; Seed: 1; Size: 6; Knapsack; Restarts: {i}; Current Restart: {j}'
#         plt.title(label=title)
#         plt.show()

# Combo 2
# for i in [5, 10, 15]:
#     for j in range(1, i+1):
#         rhcKS2 = rhc_combo2KS[(rhc_combo2KS['current_restart'] == j) & (rhc_combo2KS['Restarts'] == i)]
#         plt.plot(rhcKS2['Iteration'], rhcKS2['Fitness'])
#         plt.xlabel(xlabel='Iteration')
#         plt.ylabel(ylabel='Fitness')
#         title = f'RHC; Seed: 1; Size: 8; Knapsack; Restarts: {i}; Current Restart: {j}'
#         plt.title(label=title)
#         plt.show()

# Combo 3
# for i in [5, 10, 15]:
#     for j in range(1, i+1):
#         rhcKS3 = rhc_combo3KS[(rhc_combo3KS['current_restart'] == j) & (rhc_combo3KS['Restarts'] == i)]
#         plt.plot(rhcKS3['Iteration'], rhcKS3['Fitness'])
#         plt.xlabel(xlabel='Iteration')
#         plt.ylabel(ylabel='Fitness')
#         title = f'RHC; Seed: 1; Size: 10; Knapsack; Restarts: {i}; Current Restart: {j}'
#         plt.title(label=title)
#         plt.show()

$$ \text{Plot SA Seed Averages}
$$

In [ ]:
# FOUR PEAKS
# fig, axes = plt.subplots(nrows=1,
#                          ncols=3,
#                          figsize=(15, 5))
# for ax in axes:
#     ax.set_xlabel(xlabel='Iteration')
#     ax.set_ylabel(ylabel='Fitness')
#     title = f'sa; Seed: 1; Size: 4; Four Peaks'
#     ax.set_title(label=title)
# plt.show()

# for i in range(1, i+1):
#     saFP1 = sa_combo1FP[(sa_combo1FP['current_restart'] == j) & (sa_combo1FP['Restarts'] == 15)]        
#     plt.plot(saFP1['Iteration'], saFP1['Fitness'])
#     plt.xlabel(xlabel='Iteration')
#     plt.ylabel(ylabel='Fitness')
#     title = f'sa; Seed: 1; Size: 4; Four Peaks; Restarts: {i}; Current Restart: {j}'
#     plt.title(label=title)
#     plt.show()

# combo 2
# for i in [5, 10, 15]:
#     for j in range(1, i+1):
#         saFP2 = sa_combo2FP[(sa_combo2FP['current_restart'] == j) & (sa_combo2FP['Restarts'] == i)]        
#         plt.plot(saFP2['Iteration'], saFP2['Fitness'])
#         plt.xlabel(xlabel='Iteration')
#         plt.ylabel(ylabel='Fitness')
#         title = f'sa; Seed: 1; Size: 6; Four Peaks; Restarts: {i}; Current Restart: {j}'
#         plt.title(label=title)
#         plt.show()

# combo 3
# for i in range(1, i+1):
#     saFP3 = sa_combo3FP[(sa_combo3FP['current_restart'] == j) & (sa_combo3FP['Restarts'] == 15)]
#     plt.plot(saFP3['Iteration'], saFP3['Fitness'])
#     plt.xlabel(xlabel='Iteration')
#     plt.ylabel(ylabel='Fitness')
#     title = f'sa; Seed: 1; Size: 8; Four Peaks; Restarts: {i}; Current Restart: {j}'
#     plt.title(label=title)
#     plt.show()

# FLIP FLOP
# Combo 1
# for i in range(15):
#     saFF1 = sa_combo1FF[(sa_combo1FF['current_restart'] == i) & (sa_combo1FF['Restarts'] == 15)]
#     plt.plot(saFF1['Iteration'], saFF1['Fitness'])
#     plt.xlabel(xlabel='Iteration')
#     plt.ylabel(ylabel='Fitness')
#     title = f'sa; Seed: 1; Size: 4; Flip Flop; Restarts: 15; Current Restart: {i}'
#     plt.title(label=title)
#     plt.show()

# Combo 2
# for i in [5, 10, 15]:
#     for j in range(1, i+1):
#         saFF2 = sa_combo2FF[(sa_combo2FF['current_restart'] == j) & (sa_combo2FF['Restarts'] == i)]
#         plt.plot(saFF2['Iteration'], saFF2['Fitness'])
#         plt.xlabel(xlabel='Iteration')
#         plt.ylabel(ylabel='Fitness')
#         title = f'sa; Seed: 1; Size: 6; Flip Flop; Restarts: {i}; Current Restart: {j}'
#         plt.title(label=title)
#         plt.show()

# Combo 3
# for i in [5, 10, 15]:
#     for j in range(1, i+1):
#         saFF3 = sa_combo3FF[(sa_combo3FF['current_restart'] == j) & (sa_combo3FF['Restarts'] == i)]
#         plt.plot(saFF3['Iteration'], saFF3['Fitness'])
#         plt.xlabel(xlabel='Iteration')
#         plt.ylabel(ylabel='Fitness')
#         title = f'sa; Seed: 1; Size: 8; Flip Flop; Restarts: {i}; Current Restart: {j}'
#         plt.title(label=title)
#         plt.show()

# KNAPSACK
# Combo 1
# for i in [5, 10, 15]:
#     for j in range(1, i+1):
#         saKS1 = sa_combo1KS[(sa_combo1KS['current_restart'] == j) & (sa_combo1KS['Restarts'] == i)]
#         plt.plot(saKS1['Iteration'], saKS1['Fitness'])
#         plt.xlabel(xlabel='Iteration')
#         plt.ylabel(ylabel='Fitness')
#         title = f'sa; Seed: 1; Size: 6; Knapsack; Restarts: {i}; Current Restart: {j}'
#         plt.title(label=title)
#         plt.show()

# Combo 2
# for i in [5, 10, 15]:
#     for j in range(1, i+1):
#         saKS2 = sa_combo2KS[(sa_combo2KS['current_restart'] == j) & (sa_combo2KS['Restarts'] == i)]
#         plt.plot(saKS2['Iteration'], saKS2['Fitness'])
#         plt.xlabel(xlabel='Iteration')
#         plt.ylabel(ylabel='Fitness')
#         title = f'sa; Seed: 1; Size: 8; Knapsack; Restarts: {i}; Current Restart: {j}'
#         plt.title(label=title)
#         plt.show()

# Combo 3
# for i in [5, 10, 15]:
#     for j in range(1, i+1):
#         saKS3 = sa_combo3KS[(sa_combo3KS['current_restart'] == j) & (sa_combo3KS['Restarts'] == i)]
#         plt.plot(saKS3['Iteration'], saKS3['Fitness'])
#         plt.xlabel(xlabel='Iteration')
#         plt.ylabel(ylabel='Fitness')
#         title = f'sa; Seed: 1; Size: 10; Knapsack; Restarts: {i}; Current Restart: {j}'
#         plt.title(label=title)
#         plt.show()

$$ \text{Plot GA Seed Averages}
$$

In [ ]:
# FOUR PEAKS
# fig, axes = plt.subplots(nrows=1,
#                          ncols=3,
#                          figsize=(15, 5))
# for ax in axes:
#     ax.set_xlabel(xlabel='Iteration')
#     ax.set_ylabel(ylabel='Fitness')
#     title = f'ga; Seed: 1; Size: 4; Four Peaks'
#     ax.set_title(label=title)
# plt.show()

# for i in range(1, i+1):
#     gaFP1 = ga_combo1FP[(ga_combo1FP['current_restart'] == j) & (ga_combo1FP['Restarts'] == 15)]        
#     plt.plot(gaFP1['Iteration'], gaFP1['Fitness'])
#     plt.xlabel(xlabel='Iteration')
#     plt.ylabel(ylabel='Fitness')
#     title = f'ga; Seed: 1; Size: 4; Four Peaks; Restarts: {i}; Current Restart: {j}'
#     plt.title(label=title)
#     plt.show()

# combo 2
# for i in [5, 10, 15]:
#     for j in range(1, i+1):
#         gaFP2 = ga_combo2FP[(ga_combo2FP['current_restart'] == j) & (ga_combo2FP['Restarts'] == i)]        
#         plt.plot(gaFP2['Iteration'], gaFP2['Fitness'])
#         plt.xlabel(xlabel='Iteration')
#         plt.ylabel(ylabel='Fitness')
#         title = f'ga; Seed: 1; Size: 6; Four Peaks; Restarts: {i}; Current Restart: {j}'
#         plt.title(label=title)
#         plt.show()

# combo 3
# for i in range(1, i+1):
#     gaFP3 = ga_combo3FP[(ga_combo3FP['current_restart'] == j) & (ga_combo3FP['Restarts'] == 15)]
#     plt.plot(gaFP3['Iteration'], gaFP3['Fitness'])
#     plt.xlabel(xlabel='Iteration')
#     plt.ylabel(ylabel='Fitness')
#     title = f'ga; Seed: 1; Size: 8; Four Peaks; Restarts: {i}; Current Restart: {j}'
#     plt.title(label=title)
#     plt.show()

# FLIP FLOP
# Combo 1
# for i in range(15):
#     gaFF1 = ga_combo1FF[(ga_combo1FF['current_restart'] == i) & (ga_combo1FF['Restarts'] == 15)]
#     plt.plot(gaFF1['Iteration'], gaFF1['Fitness'])
#     plt.xlabel(xlabel='Iteration')
#     plt.ylabel(ylabel='Fitness')
#     title = f'ga; Seed: 1; Size: 4; Flip Flop; Restarts: 15; Current Restart: {i}'
#     plt.title(label=title)
#     plt.show()

# Combo 2
# for i in [5, 10, 15]:
#     for j in range(1, i+1):
#         gaFF2 = ga_combo2FF[(ga_combo2FF['current_restart'] == j) & (ga_combo2FF['Restarts'] == i)]
#         plt.plot(gaFF2['Iteration'], gaFF2['Fitness'])
#         plt.xlabel(xlabel='Iteration')
#         plt.ylabel(ylabel='Fitness')
#         title = f'ga; Seed: 1; Size: 6; Flip Flop; Restarts: {i}; Current Restart: {j}'
#         plt.title(label=title)
#         plt.show()

# Combo 3
# for i in [5, 10, 15]:
#     for j in range(1, i+1):
#         gaFF3 = ga_combo3FF[(ga_combo3FF['current_restart'] == j) & (ga_combo3FF['Restarts'] == i)]
#         plt.plot(gaFF3['Iteration'], gaFF3['Fitness'])
#         plt.xlabel(xlabel='Iteration')
#         plt.ylabel(ylabel='Fitness')
#         title = f'ga; Seed: 1; Size: 8; Flip Flop; Restarts: {i}; Current Restart: {j}'
#         plt.title(label=title)
#         plt.show()

# KNAPSACK
# Combo 1
# for i in [5, 10, 15]:
#     for j in range(1, i+1):
#         gaKS1 = ga_combo1KS[(ga_combo1KS['current_restart'] == j) & (ga_combo1KS['Restarts'] == i)]
#         plt.plot(gaKS1['Iteration'], gaKS1['Fitness'])
#         plt.xlabel(xlabel='Iteration')
#         plt.ylabel(ylabel='Fitness')
#         title = f'ga; Seed: 1; Size: 6; Knapsack; Restarts: {i}; Current Restart: {j}'
#         plt.title(label=title)
#         plt.show()

# Combo 2
# for i in [5, 10, 15]:
#     for j in range(1, i+1):
#         gaKS2 = ga_combo2KS[(ga_combo2KS['current_restart'] == j) & (ga_combo2KS['Restarts'] == i)]
#         plt.plot(gaKS2['Iteration'], gaKS2['Fitness'])
#         plt.xlabel(xlabel='Iteration')
#         plt.ylabel(ylabel='Fitness')
#         title = f'ga; Seed: 1; Size: 8; Knapsack; Restarts: {i}; Current Restart: {j}'
#         plt.title(label=title)
#         plt.show()

# Combo 3
# for i in [5, 10, 15]:
#     for j in range(1, i+1):
#         gaKS3 = ga_combo3KS[(ga_combo3KS['current_restart'] == j) & (ga_combo3KS['Restarts'] == i)]
#         plt.plot(gaKS3['Iteration'], gaKS3['Fitness'])
#         plt.xlabel(xlabel='Iteration')
#         plt.ylabel(ylabel='Fitness')
#         title = f'ga; Seed: 1; Size: 10; Knapsack; Restarts: {i}; Current Restart: {j}'
#         plt.title(label=title)
#         plt.show()

$$ \text{Plot MIMIC Seed Averages}
$$

In [ ]:
# FOUR PEAKS
# fig, axes = plt.subplots(nrows=1,
#                          ncols=3,
#                          figsize=(15, 5))
# for ax in axes:
#     ax.set_xlabel(xlabel='Iteration')
#     ax.set_ylabel(ylabel='Fitness')
#     title = f'mimic; Seed: 1; Size: 4; Four Peaks'
#     ax.set_title(label=title)
# plt.show()

# for i in range(1, i+1):
#     mimicFP1 = mimic_combo1FP[(mimic_combo1FP['current_restart'] == j) & (mimic_combo1FP['Restarts'] == 15)]        
#     plt.plot(mimicFP1['Iteration'], mimicFP1['Fitness'])
#     plt.xlabel(xlabel='Iteration')
#     plt.ylabel(ylabel='Fitness')
#     title = f'mimic; Seed: 1; Size: 4; Four Peaks; Restarts: {i}; Current Restart: {j}'
#     plt.title(label=title)
#     plt.show()

# combo 2
# for i in [5, 10, 15]:
#     for j in range(1, i+1):
#         mimicFP2 = mimic_combo2FP[(mimic_combo2FP['current_restart'] == j) & (mimic_combo2FP['Restarts'] == i)]        
#         plt.plot(mimicFP2['Iteration'], mimicFP2['Fitness'])
#         plt.xlabel(xlabel='Iteration')
#         plt.ylabel(ylabel='Fitness')
#         title = f'mimic; Seed: 1; Size: 6; Four Peaks; Restarts: {i}; Current Restart: {j}'
#         plt.title(label=title)
#         plt.show()

# combo 3
# for i in range(1, i+1):
#     mimicFP3 = mimic_combo3FP[(mimic_combo3FP['current_restart'] == j) & (mimic_combo3FP['Restarts'] == 15)]
#     plt.plot(mimicFP3['Iteration'], mimicFP3['Fitness'])
#     plt.xlabel(xlabel='Iteration')
#     plt.ylabel(ylabel='Fitness')
#     title = f'mimic; Seed: 1; Size: 8; Four Peaks; Restarts: {i}; Current Restart: {j}'
#     plt.title(label=title)
#     plt.show()

# FLIP FLOP
# Combo 1
# for i in range(15):
#     mimicFF1 = mimic_combo1FF[(mimic_combo1FF['current_restart'] == i) & (mimic_combo1FF['Restarts'] == 15)]
#     plt.plot(mimicFF1['Iteration'], mimicFF1['Fitness'])
#     plt.xlabel(xlabel='Iteration')
#     plt.ylabel(ylabel='Fitness')
#     title = f'mimic; Seed: 1; Size: 4; Flip Flop; Restarts: 15; Current Restart: {i}'
#     plt.title(label=title)
#     plt.show()

# Combo 2
# for i in [5, 10, 15]:
#     for j in range(1, i+1):
#         mimicFF2 = mimic_combo2FF[(mimic_combo2FF['current_restart'] == j) & (mimic_combo2FF['Restarts'] == i)]
#         plt.plot(mimicFF2['Iteration'], mimicFF2['Fitness'])
#         plt.xlabel(xlabel='Iteration')
#         plt.ylabel(ylabel='Fitness')
#         title = f'mimic; Seed: 1; Size: 6; Flip Flop; Restarts: {i}; Current Restart: {j}'
#         plt.title(label=title)
#         plt.show()

# Combo 3
# for i in [5, 10, 15]:
#     for j in range(1, i+1):
#         mimicFF3 = mimic_combo3FF[(mimic_combo3FF['current_restart'] == j) & (mimic_combo3FF['Restarts'] == i)]
#         plt.plot(mimicFF3['Iteration'], mimicFF3['Fitness'])
#         plt.xlabel(xlabel='Iteration')
#         plt.ylabel(ylabel='Fitness')
#         title = f'mimic; Seed: 1; Size: 8; Flip Flop; Restarts: {i}; Current Restart: {j}'
#         plt.title(label=title)
#         plt.show()

# KNAPSACK
# Combo 1
# for i in [5, 10, 15]:
#     for j in range(1, i+1):
#         mimicKS1 = mimic_combo1KS[(mimic_combo1KS['current_restart'] == j) & (mimic_combo1KS['Restarts'] == i)]
#         plt.plot(mimicKS1['Iteration'], mimicKS1['Fitness'])
#         plt.xlabel(xlabel='Iteration')
#         plt.ylabel(ylabel='Fitness')
#         title = f'mimic; Seed: 1; Size: 6; Knapsack; Restarts: {i}; Current Restart: {j}'
#         plt.title(label=title)
#         plt.show()

# Combo 2
# for i in [5, 10, 15]:
#     for j in range(1, i+1):
#         mimicKS2 = mimic_combo2KS[(mimic_combo2KS['current_restart'] == j) & (mimic_combo2KS['Restarts'] == i)]
#         plt.plot(mimicKS2['Iteration'], mimicKS2['Fitness'])
#         plt.xlabel(xlabel='Iteration')
#         plt.ylabel(ylabel='Fitness')
#         title = f'mimic; Seed: 1; Size: 8; Knapsack; Restarts: {i}; Current Restart: {j}'
#         plt.title(label=title)
#         plt.show()

# Combo 3
# for i in [5, 10, 15]:
#     for j in range(1, i+1):
#         mimicKS3 = mimic_combo3KS[(mimic_combo3KS['current_restart'] == j) & (mimic_combo3KS['Restarts'] == i)]
#         plt.plot(mimicKS3['Iteration'], mimicKS3['Fitness'])
#         plt.xlabel(xlabel='Iteration')
#         plt.ylabel(ylabel='Fitness')
#         title = f'mimic; Seed: 1; Size: 10; Knapsack; Restarts: {i}; Current Restart: {j}'
#         plt.title(label=title)
#         plt.show()